In [4]:
import time
import random
import pandas as pd
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


def build_search_url(keywords):
    # Process each keyword, replacing spaces with +, and surrounding phrases with %22
    processed_keywords = []
    for keyword in keywords:
        if " " in keyword:  # For multi-word phrases
            processed_keywords.append('%22' + keyword.replace(" ", "+") + '%22')
        else:
            processed_keywords.append(keyword.replace(" ", "+"))  # For single words, just replace spaces with +
    
    # Join the processed keywords with '+' to create a valid URL query string
    query = "+".join(processed_keywords)
    
    # Construct the search URL dynamically
    search_url = f"https://link.springer.com/search?new-search=true&query={query}&content-type=article&dateFrom=&dateTo=&language=En&taxonomy=%22Artificial+Intelligence%22&facet-sub-discipline=%22Artificial+Intelligence%22&facet-sub-discipline=%22Computer+Science%2C+general%22&sortBy=relevance"
    
    return search_url



def create_empty_table(db_name, table_name):
    # Connect to SQLite database (it will create the database file if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # SQL query to create a table with the specified columns, with 'frequency' as the first column
    create_table_query = f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            frequency INTEGER DEFAULT 1,  -- New column to track frequency of duplicates (now the first column)
            journal_name TEXT,
            journal_link TEXT,
            paper_title TEXT,
            paper_link TEXT,
            publication_date TEXT,
            publishing_model TEXT,
            journal_impact_factor TEXT,
            five_year_journal_impact_factor TEXT,
            submission_to_first_decision TEXT,
            downloads TEXT
        )
    '''

    # Execute the query to create the table
    cursor.execute(create_table_query)
    
    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print(f"Table '{table_name}' created successfully in database '{db_name}' with 'frequency' as the first column.")


def update_frequency(db_name, table_name):
    # Connect to SQLite database
    conn = sqlite3.connect(db_name)

    # Load the existing data from the SQLite database into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Check if the 'frequency' column exists in the table, if not, create it
    if 'frequency' not in df.columns:
        df['frequency'] = 1  # Add 'frequency' column with default value of 1
    else:
        print("Frequency column already exists.")

    # Group by 'journal_name' and 'journal_link' to count duplicates
    df['frequency'] = df.groupby(['journal_name', 'journal_link'])['journal_name'].transform('count')

    # Drop duplicates based on 'journal_name' and 'journal_link', keeping the first occurrence
    df = df.drop_duplicates(subset=['journal_name', 'journal_link'], keep='first')

    # Replace the existing table with the updated DataFrame
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    print(f"Frequency updated and duplicates removed in table '{table_name}'.")

    # Close the connection
    conn.close()


def add_select_column(db_name, table_name):
    # Step 1: Connect to the SQLite database
    conn = sqlite3.connect(db_name)

    # Step 2: Load the existing data from the specified table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Step 3: Create the new 'select' column with default value 0
    df.insert(0, 'select', 0)  # Insert 'select' as the first column, with default value 0

    # Step 4: Replace the existing table with the updated DataFrame
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    # Step 5: Close the connection to the database
    conn.close()

    print(f"Column 'select' added as the first column with default value 0 and table '{table_name}' updated in database '{db_name}'.")


def scrape_springer_journals(search_url, num_pages, db_name, table_name):
    # Set up the WebDriver (Chrome in this case)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # Connect to SQLite database (we assume the table is already created)
    conn = sqlite3.connect(db_name)

    # Retrieve existing journal links from the database
    existing_links = pd.read_sql_query(f"SELECT journal_link FROM {table_name}", conn)['journal_link'].tolist()

    # DataFrame to store results temporarily with updated column names (snake_case)
    columns = ['journal_name', 'journal_link', 'paper_title', 'paper_link', 'publication_date', 
               'publishing_model', 'journal_impact_factor', 'five_year_journal_impact_factor', 
               'submission_to_first_decision', 'downloads']
    df = pd.DataFrame(columns=columns)

    # Loop over the pages
    for page in range(1, num_pages + 1):
        # Format the URL for pagination
        formatted_url = f"{search_url}&page={page}"
        print(f"Scraping page {page}: {formatted_url}")
        
        # Open the URL in the browser
        driver.get(formatted_url)
        
        # Wait for the page to load dynamically
        time.sleep(5)  # Adjust if necessary based on your connection speed
        
        # Get the page source and parse it with BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Find all Sample Paper elements
        SamplePaper_elements = soup.select('li.app-card-open')  # Update to match the correct list class
        
        # Loop over each SamplePaper element and extract information
        for SamplePaper in SamplePaper_elements:
            # Extract the paper title and link
            SamplePaper_name_element = SamplePaper.select_one('.app-card-open__link')
            SamplePaper_title = SamplePaper_name_element.get_text(strip=True) if SamplePaper_name_element else 'N/A'
            SamplePaper_link = SamplePaper_name_element.get('href') if SamplePaper_name_element else 'N/A'
            
            # Append base URL if the link is relative
            if SamplePaper_link and not SamplePaper_link.startswith("http"):
                SamplePaper_link = "https://link.springer.com" + SamplePaper_link
            
            # Extract the journal name and link
            journal_name_element = SamplePaper.select_one('.u-color-inherit')
            journal_name = journal_name_element.get_text(strip=True) if journal_name_element else 'Journal name not found'
            journal_link = journal_name_element.get('href') if journal_name_element else 'N/A'
            
            # Append base URL if the journal link is relative
            if journal_link and not journal_link.startswith("http"):
                journal_link = "https://link.springer.com" + journal_link

            # Check if the journal link already exists in the database
            if journal_link in existing_links:
                print(f"Skipping {journal_name} as it's already in the database.")
                continue  # Skip to the next journal if it's already in the database

            # Extract the publication date
            publication_date_element = SamplePaper.find('span', {'data-test': 'published'})
            publication_date = publication_date_element.get_text(strip=True) if publication_date_element else 'Date not found'
            publication_date = publication_date.split()[-1]
            
            # Create a row with extracted data and fill the rest with empty values
            temp_df = pd.DataFrame({
                'journal_name': [journal_name],
                'journal_link': [journal_link],
                'paper_title': [SamplePaper_title.replace('’', "'").replace('–', '-')],
                'paper_link': [SamplePaper_link],
                'publication_date': [publication_date],
                'publishing_model': [""],  # Initially empty
                'journal_impact_factor': [""],  # Initially empty
                'five_year_journal_impact_factor': [""],  # Initially empty
                'submission_to_first_decision': [""],  # Initially empty
                'downloads': [""],  # Initially empty
            })
            
            # Concatenate the temporary DataFrame with the main DataFrame
            df = pd.concat([df, temp_df], ignore_index=True)
        
        print(f"Remained {num_pages - page} pages")

        # Wait for a random time between 5 and 25 seconds before proceeding to the next page
        random_sleep_time = random.randint(5, 25)
        time.sleep(random_sleep_time)

    # Close the browser when done
    driver.quit()

    # Drop duplicates
    df = df.drop_duplicates(keep='first')

    # Insert the DataFrame into the existing SQLite table (append the data)
    if not df.empty:
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"New data added to table '{table_name}' in database '{db_name}'")
    else:
        print("No new data to add to the database.")

    # Close the connection
    conn.close()

    print(f"Data saved to table '{table_name}' in database '{db_name}'")



def scrape_journal_details(db_name, table_name, row_limit=None):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)

    # Load the existing data from the SQLite database into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Replace empty strings with NaN (which is treated as null)
    df.replace("", None, inplace=True)

    # Filter the DataFrame to get only rows where the following columns are missing or empty
    missing_columns = ['publishing_model', 'journal_impact_factor', 'five_year_journal_impact_factor', 'submission_to_first_decision', 'downloads']
    df_filtered = df[df[missing_columns].isnull().any(axis=1)]  # Rows with missing information or empty strings

    # Apply row_limit only if it is set, otherwise process all rows
    if row_limit:
        df_filtered = df_filtered.head(row_limit)

    # Set up the WebDriver (Chrome in this case)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    total_rows = len(df_filtered)  # Get the total number of filtered rows (journals) in the DataFrame

    # Loop over each journal row in the filtered DataFrame
    for index, row in df_filtered.iterrows():
        remaining_journals = total_rows - (index + 1)  # Calculate remaining journals

        journal_name = row['journal_name']
        journal_link = row['journal_link']

        # Open the journal link
        driver.get(journal_link)

        # Random delay between 5-25 seconds
        random_sleep_time = random.randint(5, 25)
        print(f"Waiting for {random_sleep_time} seconds before scraping journal page...")
        time.sleep(random_sleep_time)

        # Get the page source and parse it with BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract publishing model
        publishing_model = ""
        publishing_model_element = soup.select_one('.app-journal-masthead__publishing-model-link')
        if publishing_model_element:
            publishing_model = publishing_model_element.get_text(strip=True)

        # Extract the journal metrics (impact factor, 5-year impact factor, etc.)
        impact_factor = ""
        five_year_impact_factor = ""
        submission_to_decision = ""
        downloads = ""

        metrics_section = soup.select_one('.app-journal-overview__metrics')

        if metrics_section:
            # Extract specific metrics
            impact_factor_element = metrics_section.find('dd', {'data-test': 'impact-factor-value'})
            if impact_factor_element:
                impact_factor = impact_factor_element.get_text(strip=True)

            five_year_impact_factor_element = metrics_section.find('dd', {'data-test': 'five-year-impact-factor-value'})
            if five_year_impact_factor_element:
                five_year_impact_factor = five_year_impact_factor_element.get_text(strip=True)

            submission_to_decision_element = metrics_section.find('dd', {'data-test': 'metrics-speed-value'})
            if submission_to_decision_element:
                submission_to_decision = submission_to_decision_element.get_text(strip=True)

            downloads_element = metrics_section.find('dd', {'data-test': 'metrics-downloads-value'})
            if downloads_element:
                downloads = downloads_element.get_text(strip=True)

        # Ensure there's data to update
        if publishing_model or impact_factor or five_year_impact_factor or submission_to_decision or downloads:
            cursor = conn.cursor()
            cursor.execute(f'''
                UPDATE {table_name}
                SET publishing_model = ?, journal_impact_factor = ?, five_year_journal_impact_factor = ?, submission_to_first_decision = ?, downloads = ?
                WHERE journal_link = ?
            ''', (publishing_model, impact_factor, five_year_impact_factor, submission_to_decision, downloads, journal_link))
            conn.commit()

        print(f"Updated details for {journal_name} (Link: {journal_link}).")

    # Close the browser and the database connection
    driver.quit()
    conn.close()

    print(f"Data updated in {db_name}")


In [6]:
# Example usage:
keywords = ["energy", "deep learning"]  # Example with a single word and multi-word phrases
search_url = build_search_url(keywords)  # Assuming you have the build_search_url function
print(search_url)

table_name = '_'.join([item.replace(' ', '') for item in keywords])

db_name = 'springer.db'

# Create the table
create_empty_table(db_name, table_name)

# Run the scraper
scrape_springer_journals(search_url=search_url, num_pages=2, db_name=db_name, table_name=table_name)

# Update the frequency for repeated entries
update_frequency(db_name, table_name)

add_select_column(db_name, table_name)


https://link.springer.com/search?new-search=true&query=energy+%22deep+learning%22&content-type=article&dateFrom=&dateTo=&language=En&taxonomy=%22Artificial+Intelligence%22&facet-sub-discipline=%22Artificial+Intelligence%22&facet-sub-discipline=%22Computer+Science%2C+general%22&sortBy=relevance
Column 'select' added as the first column with default value 0 and table 'energy_deeplearning' updated in database 'springer.db'.


In [8]:
scrape_journal_details(db_name, table_name, row_limit=None)

Waiting for 21 seconds before scraping journal page...
Updated details for Neural Computing and Applications (Link: https://link.springer.com/journal/521).
Waiting for 8 seconds before scraping journal page...
Updated details for SN Computer Science (Link: https://link.springer.com/journal/42979).
Waiting for 11 seconds before scraping journal page...
Updated details for Artificial Intelligence Review (Link: https://link.springer.com/journal/10462).
Waiting for 25 seconds before scraping journal page...
Updated details for Applied Intelligence (Link: https://link.springer.com/journal/10489).
Waiting for 11 seconds before scraping journal page...
Updated details for Telecommunication Systems (Link: https://link.springer.com/journal/11235).
Waiting for 13 seconds before scraping journal page...
Updated details for Computing (Link: https://link.springer.com/journal/607).
Waiting for 18 seconds before scraping journal page...
Updated details for The Journal of Supercomputing (Link: https:/